In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

Agrupando os ações por setor de atuação

In [2]:
df_hist_cot_norm = pd.read_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_hist_cot_norm.csv")
df_hist_cot_norm["Date"] = pd.to_datetime(df_hist_cot_norm["Date"])
df_hist_cot_norm.set_index("Date", inplace=True)

In [3]:
# Listando os setores de mercado
df_setores = pd.read_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\info_empresas.csv")[["tickers","setor"]]
df_setores.set_index("tickers", inplace=True)
lista_setores = list(set(df_setores["setor"]))

In [4]:
for setor in lista_setores:
    
    df_setor = pd.DataFrame(index=df_hist_cot_norm.index)
    
    for ticker in df_hist_cot_norm.columns:
    
        if df_setores.loc[ticker]["setor"] == setor:
            
            df_setor[ticker] = df_hist_cot_norm[ticker]
    
    df_setor.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\hist_cot_norm_setores\\{setor}.csv")

Gerando um df com o histórico de variação média de cada setor

In [5]:
data_inicial = '2013-01-01'
data_final = '2023-10-01'
datas = pd.date_range(start=data_inicial, end=data_final, freq='D')
df_media_setores = pd.DataFrame({'Date': datas}).set_index('Date')

for setor in lista_setores:

    df_hist_cot_ticker = pd.read_csv(
        f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\hist_cot_norm_setores\\{setor}.csv", 
        index_col="Date")
    
    for data in df_hist_cot_ticker.index:
        
        soma_linha = 0
        cont_tickers_somados = 0
        
        for ticker in df_hist_cot_ticker.columns:
            
            cond_calc_media = df_hist_cot_ticker.loc[data, ticker] !=0 and not np.isnan(df_hist_cot_ticker.loc[data, ticker])
            if cond_calc_media:
                
                soma_linha += df_hist_cot_ticker.loc[data, ticker]
                cont_tickers_somados += 1
        
        if cont_tickers_somados > 0:
            df_hist_cot_ticker.loc[data, "media"] = soma_linha / cont_tickers_somados 
    
    for data in df_hist_cot_ticker.index:
        df_media_setores.loc[data, setor] = df_hist_cot_ticker.loc[data, "media"]

Trantando valores faltantes

In [6]:
# Removendo datas sem cotação (finais de semana, feriados, ...)
df_media_setores.dropna(inplace=True, how="all")

# Tratando datas anteriores à data de referencia (data inicial de amostragem)
for setor in df_media_setores.columns:

    data_ref = df_media_setores[setor].first_valid_index()

    for data in df_media_setores.index:

        if data < data_ref:

            df_media_setores.loc[data, setor] = 0

# Tratando datas com cotações faltantes
for setor in df_media_setores.columns:
    df_media_setores[f"{setor}"].ffill(inplace=True)

Exportando os histórios das médias das cotações normalizadas de cada setor

In [7]:
df_media_setores.to_csv(f"C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_media_setores.csv", index=True)

Decompondo as séries temporais das médias das cotações normalizadas de cada setor

In [8]:
df_tend_media_setores = pd.DataFrame(index = df_media_setores.index)
df_saz_media_setores = pd.DataFrame(index = df_media_setores.index)
df_resid_media_setores = pd.DataFrame(index = df_media_setores.index)

for setor in df_media_setores.columns:

    # 22 dias de cotação por mês / 252 dias de cotação por ano / df_media_segm.shape[0]//2 maior período possível para decomposição da base de dados
    
    df_tend_media_setores[f"{setor}"] = seasonal_decompose(df_media_setores[f"{setor}"],
                                                period = 252,
                                                model = "additive").trend
    df_saz_media_setores[f"{setor}"] = seasonal_decompose(df_media_setores[f"{setor}"],
                                                period = 252,
                                                model = "additive").seasonal
    df_resid_media_setores[f"{setor}"] = seasonal_decompose(df_media_setores[f"{setor}"],
                                                period = 252,
                                                model = "additive").resid

df_tend_media_setores.dropna(inplace=True)
df_saz_media_setores.dropna(inplace=True)
df_resid_media_setores.dropna(inplace=True)

In [9]:
df_tend_media_setores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_tend_media_setores.csv")
df_saz_media_setores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_saz_media_setores.csv")
df_resid_media_setores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\df_resid_media_setores.csv")

Gerando a matriz de correlação entre os setores

In [ ]:
mtx_corr_setores = df_media_setores.corr()
mtx_corr_setores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\mtx_corr_setores.csv")

Gerando a matriz de correlação entre as sazonalidades da médias dos setores

In [ ]:
mtx_corr_saz_setores = df_saz_media_setores.corr()
mtx_corr_saz_setores.to_csv("C:\\Users\\victo\\OneDrive\\Documentos\\github\\tcc-analise-setores-mercado-financeiro\\reports\\mtx_corr_saz_setores.csv")